# Part 1

In [44]:
import numpy as np

In [45]:
puzzle_input = '220D69802BE00A0803711E1441B1006E39C318A12730C200DCE66D2CCE360FA0055652CD32966E3004677EDF600B0803B1361741510076254138D8A00E4FFF3E3393ABE4FC7AC10410010799D2A4430003764DBE281802F3102CA00D4840198430EE0E00021D04E3F41F84AE0154DFDE65A17CCBFAFA14ADA56854FE5E3FD5BCC53B0D2598027A00848C63F2B918C7E513DEC3290051B3867E009CCC5FE46BD520007FE5E8AD344B37583D0803E40085475887144C01A8C10FE2B9803B0720D45A3004652FD8FA05F80122CAF91E5F50E66BEF8AB000BB0F4802039C20917B920B9221200ABF0017B9C92CCDC76BD3A8C4012CCB13CB22CDB243E9C3D2002067440400D9BE62DAC4D2DC0249BF76B6F72BE459B279F759AE7BE42E0058801CC059B08018A0070012CEC045BA01006C03A8000D46C02FA000A8EA007200800E00618018E00410034220061801D36BF178C01796FC52B4017100763547E86000084C7E8910AC0027E9B029FE2F4952F96D81B34C8400C24AA8CDAF4F1E98027C00FACDE3BA86982570D13AA640195CD67B046F004662711E989C468C01F1007A10C4C8320008742287117C401A8C715A3FC2C8EB3777540048272DFE7DE1C0149AC8BC9E79D63200B674013978E8BE5E3A2E9AA3CCDD538C01193CFAB0A146006AA00087C3E88B130401D8E304A239802F39FAC922C0169EA3248DF2D600247C89BCDFE9CA7FFD8BB49686236C9FF9795D80C0139BEC4D6C017978CF78C5EB981FCE7D4D801FA9FB63B14789534584010B5802F3467346D2C1D1E080355B00424FC99290C7E5D729586504803A2D005E677F868C271AA479CEEB131592EE5450043A932697E6A92C6E164991EFC4268F25A294600B5002A3393B31CC834B972804D2F3A4FD72B928E59219C9C771EC3DC89D1802135C9806802729694A6E723FD6134C0129A019E600'

In [46]:
hex_str_dict = {
    "0": "0000",
    "1": "0001",
    "2": "0010",
    "3": "0011",
    "4": "0100",
    "5": "0101",
    "6": "0110",
    "7": "0111",
    "8": "1000",
    "9": "1001",
    "A": "1010",
    "B": "1011",
    "C": "1100",
    "D": "1101",
    "E": "1110",
    "F": "1111"
}

In [47]:
# Could be done with out of the box tools
# But not sure if the same results
def hex_to_binary(hex_str):
    
    return ("").join([hex_str_dict[char] for char in hex_str])

In [48]:
binary_str = hex_to_binary(puzzle_input)

In [49]:
def discover_hierarchy(packet):
        
    version, packet= parse_packet(packet,0)
    print(version)

In [50]:
def parse_packet(packet, version_num):

    if packet:

        version = int(packet[:3],2)
        version_num += version

        type_id = int(packet[3:6],2)

        packet = packet[6:]

        if type_id == 4:
            literal = ''

            for i in range(0,len(packet),5):

                chunk = packet[i:i+5]
                # If chunk starts with 1 then not last
                if packet[i] == '1':
                    literal = literal + chunk[1:]  

                # assume it's a 0, so we're at the end
                else:
                    literal = literal + chunk[1:]  
                    # Don't execute anything after
                    break
            if packet:
                return packet[i+5:],version_num
            else:
                return packet,version_num

        # Only other option but just to be safe
        # We're at an operator
        else:

            length_type_id = packet[0]

            if length_type_id == '0':
                length = int(packet[1:16],2)
                packet = packet[16:]

                length_attempted = 0
                
                while length_attempted < length:
                    start_len = len(packet)
                    packet,version_num = parse_packet(packet,version_num)
                    end_len = len(packet)
                    length_attempted += start_len - end_len

            # assume it's legnth ID 1
            elif length_type_id == '1':
                num_subpackets = int(packet[1:12],2)
                packet = packet[12:]
                
                subpackets_completed = 0
                while subpackets_completed < num_subpackets:
                    subpackets_completed += 1
                    packet,version_num = parse_packet(packet,version_num)

 
    return packet,version_num

In [51]:
def total_version_nums(puzzle_input):
    
    binary_input = hex_to_binary(puzzle_input)
    
    parsed_packet,version_sum = parse_packet(binary_input,0)
    
    return version_sum

In [52]:
total_version_nums(puzzle_input)

1002

# Part 2

In [100]:
def operate(operator, vals):
    if operator == 0:
        print('sum')
        return [sum(vals)]
    if operator == 1:
        print('product')
        return [np.prod(vals)]
    if operator == 2:
        return [min(vals)]
    if operator == 3:
        return [max(vals)]
    if operator == 5:
        return [int(vals[0] > vals[1])]
    if operator == 6:
        return [int(vals[0] < vals[1])]
    if operator == 7:
        print('equals')
        return [int(vals[0] == vals[1])]

In [103]:
def parse_packet_for_value(packet,subvals):
    print()
    print("packet:",packet)
    print(subvals)
    if packet:

        type_id = int(packet[3:6],2)

        packet = packet[6:]

        if type_id == 4:
            literal = ''

            for i in range(0,len(packet),5):

                chunk = packet[i:i+5]
                # If chunk starts with 1 then not last
                if packet[i] == '1':
                    literal = literal + chunk[1:]  

                # assume it's a 0, so we're at the end
                else:
                    literal = literal + chunk[1:]  
                    value = int(literal,2)
                    # Don't execute anything after
                    break
            if packet:
                return packet[i+5:],subvals+[value]
            else:
                return packet,subvals+[value]

        # Only other option but just to be safe
        # We're at an operator
        else:
                
            length_type_id = packet[0]

            if length_type_id == '0':

                length = int(packet[1:16],2)
                print("length:",length)
                packet = packet[16:]

                length_attempted = 0
                
                while length_attempted < length:
                    start_len = len(packet)
                    packet,subvals = parse_packet_for_value(packet,subvals)
                    end_len = len(packet)
                    length_attempted += start_len - end_len
                subvals = operate(type_id,subvals)


            # assume it's legnth ID 1
            elif length_type_id == '1':
                num_subpackets = int(packet[1:12],2)
                packet = packet[12:]
                print("num subpackets:",num_subpackets)
                
                subpackets_completed = 0
                while subpackets_completed < num_subpackets:
                    print('going through subpackets')
                    subpackets_completed += 1
                    packet,subvals = parse_packet_for_value(packet,subvals)
                subvals = operate(type_id,subvals)
 
    return packet,subvals

In [104]:
parse_packet_for_value(hex_to_binary('9C0141080250320F1802104A08'),[])


packet: 10011100000000010100000100001000000000100101000000110010000011110001100000000010000100000100101000001000
[]
length: 80

packet: 0100001000000000100101000000110010000011110001100000000010000100000100101000001000
[]
num subpackets: 2
going through subpackets

packet: 0101000000110010000011110001100000000010000100000100101000001000
[]
going through subpackets

packet: 10010000011110001100000000010000100000100101000001000
[1]
sum

packet: 110001100000000010000100000100101000001000
[4]
num subpackets: 2
going through subpackets

packet: 000100000100101000001000
[4]
going through subpackets

packet: 0101000001000
[4, 2]
product
equals


IndexError: list index out of range

```
C200B40A82 finds the sum of 1 and 2, resulting in the value 3.
04005AC33890 finds the product of 6 and 9, resulting in the value 54.
880086C3E88112 finds the minimum of 7, 8, and 9, resulting in the value 7.
CE00C43D881120 finds the maximum of 7, 8, and 9, resulting in the value 9.
D8005AC2A8F0 produces 1, because 5 is less than 15.
F600BC2D8F produces 0, because 5 is not greater than 15.
9C005AC2F8F0 produces 0, because 5 is not equal to 15.
9C0141080250320F1802104A08 produces 1, because 1 + 3 = 2 * 2.
```